# Reforumlated Markowitz Model - Optimization Codes through Quantum Computing

Import the necessary libraries

In [69]:
import numpy as np
import pandas as pd
from dimod import ConstrainedQuadraticModel, Integer, Binary
import math
import time

Importing the Data

In [70]:
df = pd.read_excel("Cleaned_Data_Index_1.xlsx")

Changing the Date to the index

In [71]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Calculating the expected returns and the variance covariance matrix

In [72]:
# Calculate expected profits for each asset
expected_profits = df.mean()

# Calculate the variance-covariance matrix
cov_matrix = df.cov()


#Converting the expected profits to a numpy array
expected_profits = expected_profits.values

#The number of assets
n_assets = len(expected_profits)


### Setting the limits of the constraints

Setting the other constraints limits

In [73]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 10

# Maximum units of each asset that can be purchased (D)
D = 30

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 20

Creating the Decision Variables

In [74]:
y = [Binary(f"y_{i}") for i in range(n_assets)] #To represent y

In [75]:
X = [Integer(f"x_{i}") for i in range(n_assets)] #To represent x


Create the CQM Object

In [76]:
cqm = ConstrainedQuadraticModel()

Creating the Objective Function

In [77]:
H_obj = 0
for i in range(n_assets):
    H_obj -= expected_profits[i] * X[i]

Creating the risk constraint

In [78]:
# Convert the DataFrame to a NumPy array
cov_matrix_np = cov_matrix.to_numpy()

# Convert X to a NumPy array 
X_numeric = np.array(X)  

# Calculate the total variance
total_variance = np.dot(X_numeric.T, np.dot(cov_matrix_np, X_numeric))

Setting the Objective Function

In [79]:
cqm.set_objective(H_obj)

Setting the Risk Constraint

In [80]:
cqm.add_constraint_from_model(total_variance, '<=', R**2, "Variance", weight=1)

'Variance'

Adding the Linking Constraints

In [81]:
# Diversification Constraints
for i in range(n_assets):
    cqm.add_constraint_from_model(X[i] - D*y[i], '<=', 0, label=f"Linking_{i}", weight=1)

Adding the cardinality constraint

In [82]:
cqm.add_constraint_from_model(sum(y), '<=', K, label=f"Cardinality", weight=1)

'Cardinality'

Solving the optimization problem

In [83]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token="DEV-2559c1545adea2a38a589d00b09592efde0e1bd5")  

In [84]:
# Time the sampler
start_time = time.time()
sampleset = sampler.sample_cqm(cqm)
time_taken = time.time() - start_time

In [85]:
print(sampleset.first) 

Sample(sample={'x_0': 0.0, 'x_1': 0.0, 'x_10': 0.0, 'x_100': 0.0, 'x_101': 0.0, 'x_102': 0.0, 'x_103': 7.0, 'x_104': 0.0, 'x_105': 0.0, 'x_106': 0.0, 'x_107': 0.0, 'x_108': 0.0, 'x_109': 0.0, 'x_11': 0.0, 'x_110': 0.0, 'x_111': 0.0, 'x_112': 0.0, 'x_113': 0.0, 'x_114': 15.0, 'x_115': 6.0, 'x_116': 0.0, 'x_117': 0.0, 'x_118': 0.0, 'x_119': 0.0, 'x_12': 10.0, 'x_120': 29.0, 'x_121': 0.0, 'x_122': 0.0, 'x_123': 0.0, 'x_124': 0.0, 'x_125': 0.0, 'x_126': 0.0, 'x_127': 0.0, 'x_128': 29.0, 'x_129': 0.0, 'x_13': 6.0, 'x_130': 0.0, 'x_131': 0.0, 'x_132': 0.0, 'x_133': 0.0, 'x_134': 0.0, 'x_135': 0.0, 'x_136': 0.0, 'x_137': 0.0, 'x_138': 0.0, 'x_139': 0.0, 'x_14': 0.0, 'x_140': 0.0, 'x_141': 0.0, 'x_142': 30.0, 'x_143': 0.0, 'x_144': 0.0, 'x_15': 0.0, 'x_16': 0.0, 'x_17': 0.0, 'x_18': 0.0, 'x_19': 0.0, 'x_2': 0.0, 'x_20': 0.0, 'x_21': 0.0, 'x_22': 23.0, 'x_23': 0.0, 'x_24': 0.0, 'x_25': 7.0, 'x_26': 0.0, 'x_27': 0.0, 'x_28': 0.0, 'x_29': 0.0, 'x_3': 0.0, 'x_30': 7.0, 'x_31': 0.0, 'x_32': 0.0, 'x

Checking for the violation of constraints

In [86]:
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)

Recreating the original decision variables

In [87]:
sample = sampleset.first[0]


# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])

decision_variables


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 10.0,
 6.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 23.0,
 0.0,
 0.0,
 7.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 18.0,
 30.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 19.0,
 28.0,
 0.0,
 0.0,
 10.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 30.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 13.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 15.0,
 6.0,
 0.0,
 0.0,
 0.0,
 0.0,
 29.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 29.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 30.0,
 0.0,
 0.0]

Calculating the total profit

In [88]:
# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_profits, decision_variables))

print(total)

0.3680312091725539


Now let us test to see if all the constraints are satisfied

In [92]:
# Extract y values from sample
y_optimized = np.array([sample[f'y_{i}'] for i in range(n_assets)])

# Convert decision_variables to a numpy array for easy manipulation
x_optimized = np.array(decision_variables)

# Now, checking the constraints with these values
# Constraint 1: Risk constraint
risk_value = np.sqrt(np.dot(np.dot(x_optimized.T, cov_matrix), x_optimized))
print(f"Risk Constraint Satisfied: {risk_value <= R}")
print(f"Risk Value: {risk_value} <= {R}")

# Constraint 2: Linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}")


Risk Constraint Satisfied: True
Risk Value: 9.938048554653376 <= 10
Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 20 <= 20
Non-negativity Constraints Satisfied: True


Checking the assets to invest in

In [93]:
values_list = x_optimized.tolist()

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Asset': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Asset'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)

   Asset  Value
0    AIZ   10.0
1    AJG    6.0
2   AMGN   23.0
3   ANET    7.0
4    APH    7.0
5    BAC   20.0
6    BMY   18.0
7     BR   30.0
8    CAG   19.0
9    CAH   28.0
10  CBOE   10.0
11   CMS   30.0
12   CPB   13.0
13  CTVA   25.0
14    DG    7.0
15  FANG   15.0
16   GLW    6.0
17   JNJ   29.0
18   MRK   29.0
19   WMT   30.0


Converting the dataframe into an excel

In [496]:
# new_df.to_excel("DF1.xlsx")

Saving the Results

In [497]:
# B = [time_taken, risk_value, R, total]
# A.append(B)

# # Create a DataFrame with the specified column names
# df = pd.DataFrame(A, columns=['time_taken', 'risk_value', 'R', 'total'])

# df.to_excel("Quantum_Results.xlsx")